# A notebook to see what the data pulling process is like



As we're constructing our database we need to store this information for users

```
[
  {
    "user_id": "0000",
    "favorited": [
          {favorited_status_user_id:"", status_id:"", timestamp:""},
          {favorited_status_user_id:"", status_id:"", timestamp:""},
    ],
    "reblogged": [
         {reblogged_status_user_id:"", status_id:"", timestamp:""},
         {reblogged_status_user_id:"", status_id:"", timestamp:""},
    ],
    "following": [
         {followed_user_id:""},
         {followed_user_id:""},
    ],
    "user_status": [
         {status_id:"", timestamp:""},
         {status_id:"", timestamp:""},
    ],
  }
]
```


And to do recommendation we want it to be in this formatting:
The interaction is based on the public timeline

```
[(user_id, item_id, “did you interact”), (user_id, item_id, “did you interact”) ]
```

In [4]:
import pandas as pd
import requests as re
import time
import os
import sqlite3
import json
import random
from mastodon import Mastodon


In [5]:

mastodon = Mastodon(api_base_url = 'https://hci.social', access_token = 'rec-sys.secret')

first_twenty = pd.DataFrame(mastodon.timeline(limit=5))
first_twenty

#first_twenty.to_csv("first_twenty.csv")

## Start this from the local instance-level timeline!


Questions I have:
- Local only? Only posts from hci.social?
- a random Brian Keegan status: 111248234655094416


In [52]:
# this is a random Brian K status on 10-17
min_id = "111248234655094416"

# this is a random Casey status on 10-20
# max_id = "111267830938365471"

## this is a BK post form 10-19
max_id = "111261938877438883"

In [53]:
req_stem = "https://hci.social/api/v1/timelines/public?limit=40&min_id={min_id}&max_id={max_id}"
req_query = req_stem.format(min_id=min_id, max_id=max_id)
r = re.get(req_query)

In [54]:
public_timeline = r.json() if r.status_code == 200 else []

In [55]:
print(len(public_timeline))

print(public_timeline[0]["id"])

print(public_timeline[1]["id"])

diff = int(public_timeline[0]["id"]) - int(public_timeline[1]["id"])
print(diff)

this_timeline_snippet = pd.DataFrame(public_timeline)
this_timeline_snippet

40
111248257657169366
111248256555355749
1101813617


,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,edited_at,content,reblog,account,media_attachments,mentions,tags,emojis,card,poll
0,111248257657169366,2023-10-17T03:34:57.000Z,None,None,False,,public,en,https://mastodon.sdf.org/users/kayakpatrick/st...,https://mastodon.sdf.org/@kayakpatrick/1112482...,...,None,"<p>Some <a href=""https://mastodon.sdf.org/tags...",None,"{'id': '110950377904895722', 'username': 'kaya...","[{'id': '111251166100172260', 'type': 'image',...",[],"[{'name': 'bloomscrolling', 'url': 'https://hc...",[],None,None
1,111248256555355749,2023-10-17T03:34:36.000Z,None,None,False,,public,en,https://social.afront.org/users/MLE_online/sta...,https://social.afront.org/@MLE_online/11124825...,...,None,<p>Look at this guy being cute</p>,None,"{'id': '109521727241081569', 'username': 'MLE_...","[{'id': '111248256446005313', 'type': 'image',...",[],[],[],None,None
2,111248256329021444,2023-10-17T03:34:36.000Z,111247202420957374,109265379324797869,False,,public,en,https://fosstodon.org/users/philbaker1/statuse...,https://fosstodon.org/@philbaker1/111248256283...,...,None,"<p><a href=""https://www.actionnews5.com/2023/1...",None,"{'id': '109265379324797869', 'username': 'phil...",[],[],[],[],{'url': 'https://www.actionnews5.com/2023/10/1...,None
3,111248255273623621,2023-10-17T03:34:20.000Z,None,None,False,,public,en,https://better.boston/users/schlichtman/status...,https://better.boston/@schlichtman/11124825528...,...,None,<p>Approaching the Special Town Meeting and th...,None,"{'id': '109382342939136848', 'username': 'schl...",[],[],[],[],{'url': 'http://blog.schlichtman.org/2023/10/1...,None
4,111248254963856438,2023-10-17T03:34:15.000Z,None,None,False,,public,en,https://c.im/users/cdarwin/statuses/1112482549...,https://c.im/@cdarwin/111248254902013135,...,None,<p>Biden to Visit Israel as U.S. Struggles to ...,None,"{'id': '109332718102014928', 'username': 'cdar...",[],[],[],[],{'url': 'https://www.wsj.com/livecoverage/isra...,None
5,111248254743962471,2023-10-17T03:34:10.000Z,None,None,False,,public,en,https://mastodon.cloud/users/slashdot/statuses...,https://mastodon.cloud/@slashdot/1112482546311...,...,None,<p>'World's First Off-Road Solar SUV' Just Dro...,None,"{'id': '109458407608088142', 'username': 'slas...",[],[],[],[],{'url': 'https://tech.slashdot.org/story/23/10...,None
6,111248253542832154,2023-10-17T03:33:50.000Z,None,None,False,,public,en,https://m.ai6yr.org/users/npr_bot/statuses/111...,https://m.ai6yr.org/@npr_bot/111248253306853117,...,None,<p>NPR: Belgian police hunted Tuesday for a gu...,None,"{'id': '110188143252407607', 'username': 'npr_...","[{'id': '111248253463214785', 'type': 'image',...",[],"[{'name': 'News', 'url': 'https://hci.social/t...",[],None,None
7,111248253463855235,2023-10-17T03:33:51.000Z,None,None,False,,public,en,https://mastodon.social/users/blogdiva/statuse...,https://mastodon.social/@blogdiva/111248253336...,...,None,<p>IN OTHER NEWS <br>am feeling better from su...,None,"{'id': '110644835111706421', 'username': 'blog...",[],[],"[{'name': 'fuckcancer', 'url': 'https://hci.so...",[],None,None
8,111248253362316859,2023-10-17T03:33:51.000Z,None,None,True,"Lawn maintenance, long rambling",public,en,https://scintilla.social/users/stibbons/status...,https://scintilla.social/@stibbons/11124825335...,...,None,<p>So back in July I had tall fescue turf laid...,None,"{'id': '109266000390092934', 'username': 'stib...",[],[],[],[],None,None
9,111248252982452814,2023-10-17T03:33:43.000Z,None,None,False,,public,en,https://mstdn.social/users/_L1vY_/statuses/111...,https://mstdn.social/@_L1vY_/111248252854910109,...,None,"<p>Via Laura Burkhardt<br><span class=""h-card""...",None,"{'id': '109258141406183320', 'username': '_L1v...",[],[],"[{'name': 'leaked', 'url': 'https://hci.social...",[],None,None


'111248257657169366'

In [42]:
req_query = req_stem.format(min_id=this_timeline_snippet["id"].iloc[0])
r = re.get(req_query)

public_timeline_1 = r.json() if r.status_code == 200 else []

In [45]:
next_timeline_snippet = pd.DataFrame(public_timeline_1)
next_timeline_snippet

,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,edited_at,content,reblog,account,media_attachments,mentions,tags,emojis,card,poll
0,111248275846265908,2023-10-17T03:39:31.000Z,None,None,False,,public,en,https://mastodon.online/users/brainsevolve/sta...,https://mastodon.online/@brainsevolve/11124827...,...,None,<p>Logrunners are unusual small songbirds that...,None,"{'id': '109317206781464708', 'username': 'brai...","[{'id': '111248275676858572', 'type': 'image',...",[],"[{'name': 'WildOz', 'url': 'https://hci.social...",[],None,None
1,111248274623107219,2023-10-17T03:39:16.000Z,None,None,False,,public,en,https://mastodon.online/users/shayz0rz/statuse...,https://mastodon.online/@shayz0rz/111248274650...,...,2023-10-17T04:05:36.000Z,"<p>Not Colbert calling the at-home tests ""reli...",None,"{'id': '109723198470799100', 'username': 'shay...",[],[],[],[],None,None
2,111248274381350731,2023-10-17T03:39:09.000Z,None,None,False,,public,en,https://historians.social/users/ERBeckman/stat...,https://historians.social/@ERBeckman/111248274...,...,None,"<p>Today's <a href=""https://historians.social/...",None,"{'id': '109428811748527514', 'username': 'ERBe...","[{'id': '111248274251415171', 'type': 'image',...",[],"[{'name': 'teachworldhistory', 'url': 'https:/...",[],{'url': 'https://www.metmuseum.org/art/collect...,None
3,111248272774598255,2023-10-17T03:38:46.000Z,None,None,False,,public,en,https://chaos.social/users/gsuberland/statuses...,https://chaos.social/@gsuberland/1112482727167...,...,None,<p>tempted to be weird and make ramen for brea...,None,"{'id': '109249159921762214', 'username': 'gsub...",[],[],[],[],None,None
4,111248272409022783,2023-10-17T03:38:41.422Z,None,None,False,,public,None,https://chat.zlatiah-no.one/notes/9kxc9tvyi12r...,https://chat.zlatiah-no.one/notes/9kxc9tvyi12r...,...,2023-10-17T13:46:16.555Z,"<p><span>Have to say the </span><a href=""https...",None,"{'id': '110262276035535898', 'username': 'zlat...",[],[],"[{'name': 'megacrit', 'url': 'https://hci.soci...",[],{'url': 'https://megacrit.itch.io/dancing-duel...,None
5,111248272059357206,2023-10-17T03:38:36.000Z,None,None,False,,public,en,https://mstdn.social/users/hkrn/statuses/11124...,https://mstdn.social/@hkrn/111248272004720386,...,None,<p>Butt Plug Will Allow Toyota to Build 900-Mi...,None,"{'id': '109394712877962304', 'username': 'hkrn...",[],[],[],[],{'url': 'https://jalopnik.com/toyota-says-this...,None
6,111248271699908923,2023-10-17T03:38:30.000Z,None,None,False,,public,en,https://hachyderm.io/users/liztai/statuses/111...,https://hachyderm.io/@liztai/111248271615802946,...,None,<p>I always feel rather heartened that so many...,None,"{'id': '109487246167355535', 'username': 'lizt...",[],[],"[{'name': 'cdramas', 'url': 'https://hci.socia...",[],{'url': 'https://serahadegbite.wordpress.com/2...,None
7,111248271067710884,2023-10-17T03:38:20.000Z,None,None,False,,public,en,https://mastodon.social/users/levelbot/statuse...,https://mastodon.social/@levelbot/111248271013...,...,None,<p>we're seeing broccoli levels at about 42%</...,None,"{'id': '109348726503407410', 'username': 'leve...",[],[],[],[],None,None
8,111248271053123950,2023-10-17T03:38:21.000Z,None,None,False,,public,en,https://climatejustice.rocks/users/Brad_Rosenh...,https://climatejustice.rocks/@Brad_Rosenheim/1...,...,None,<p>Could you imagine if every time someone was...,None,"{'id': '109364152645338787', 'username': 'Brad...",[],[],"[{'name': 'swedenbelgium', 'url': 'https://hci...",[],{'url': 'https://www.espn.com/soccer/story/_/i...,None
9,111248270831385257,2023-10-17T03:38:17.000Z,None,None,False,,public,en,https://kolektiva.social/users/ejstacey/status...,https://kolektiva.social/@ejstacey/11124827080...,...,None,<p>I wrote a blog post about setting up variou...,None,"{'id': '109374040961558401', 'username': 'ejst...",[],"[{'id': '110495646908309933', 'username': 'tsl...",[],[],{'url': 'https://blog.joyrex.net/the-silent-li..

## actually write the while loop over here!

Pseudocode:


- pick a starting point
- get the list ( the first element is most recent!) 
    - Does that become the min?
    



In [64]:

# this is a random Brian K status on 10-17
min_id = "111248234655094416"


# this is a random Casey status on 10-20
# max_id = "111267830938365471"

## this is a BK post form 10-19
max_id = "111261938877438883"


timeline_df = pd.DataFrame()


req_stem = "https://hci.social/api/v1/timelines/public?limit=40&min_id={min_id}&max_id={max_id}"
req_query = req_stem.format(min_id=min_id, max_id=max_id)
r = re.get(req_query)

public_timeline = r.json() if r.status_code == 200 else []


ii = 0

while len(public_timeline) > 0:
    # keep going until you 
    
    this_TL_chunk_df = pd.DataFrame(public_timeline)
    
    
    # TODO: Don't concat these becayse we don't want huge CSVs! 
    # the newest thing is at the top, so concat old to the new
    timeline_df = pd.concat([this_TL_chunk_df, timeline_df], axis=0)
    
    # find out your new min_id
    min_id = this_TL_chunk_df["id"].max()
    
    if ii % 10 == 0:
        print("caching results") 
        this_TL_chunk_df.to_csv("data/sample_little_chunk_{}.csv".format(ii), index=False)
    
    
    
    ii += 1
    
    print("I am so proud of you, the new min id is {}".format(min_id))
    
    # don't get booted off the API
    time.sleep(1)
    
    # make another call
    
    req_query = req_stem.format(min_id=min_id, max_id=max_id)
    r = re.get(req_query)

    public_timeline = r.json() if r.status_code == 200 else []

    

print("DONE! You did so good!!")


timeline_df.to_csv("data/sample_timeline_full.csv", index=False)


caching results
I am so proud of you, the new min id is 111248257657169366
I am so proud of you, the new min id is 111248275846265908
I am so proud of you, the new min id is 111248299082352431
I am so proud of you, the new min id is 111248317555087039
I am so proud of you, the new min id is 111248340629999593
I am so proud of you, the new min id is 111248356847187061
I am so proud of you, the new min id is 111248370627403298
I am so proud of you, the new min id is 111248386797784581
I am so proud of you, the new min id is 111248409562832926
I am so proud of you, the new min id is 111248428931689008
caching results
I am so proud of you, the new min id is 111248458124020586
I am so proud of you, the new min id is 111248481849599790
I am so proud of you, the new min id is 111248500644660271
I am so proud of you, the new min id is 111248517186430660
I am so proud of you, the new min id is 111248542847018276
I am so proud of you, the new min id is 111248566807800865
I am so proud of you, th

In [65]:
timeline_df

,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,content,reblog,account,media_attachments,mentions,tags,emojis,card,poll,application
0,111251956074997131,2023-10-17T19:15:30.000Z,None,None,False,,public,en,https://beige.party/users/alexpsmith/statuses/...,https://beige.party/@alexpsmith/11125195608847...,...,<p>I find it astonishing that we have the abil...,None,"{'id': '110847987024853143', 'username': 'alex...","[{'id': '111252705013505085', 'type': 'image',...",[],[],[],None,None,NaN
1,111251955701637468,2023-10-17T19:15:24.000Z,None,None,False,,public,en,https://mastodon.org.uk/users/barcamplondon/st...,https://mastodon.org.uk/@barcamplondon/1112519...,...,<p>Today's visit to the venue went really well...,None,"{'id': '110750886544045928', 'username': 'barc...","[{'id': '111256302874886510', 'type': 'image',...",[],"[{'name': 'bclxii', 'url': 'https://hci.social...",[],{'url': 'https://twelve.barcamplondon.org/venu...,None,NaN
2,111251955573387261,2023-10-17T19:15:22.000Z,None,None,False,,public,en,https://mastodon.social/users/karlchristiankru...,https://mastodon.social/@karlchristiankrumphol...,...,<p>I did a thing: At Cartoon Crossroads Columb...,None,"{'id': '109824864543997375', 'username': 'karl...",[],[],[],[],{'url': 'https://www.youtube.com/watch?si=AsY8...,None,NaN
3,111251955129978790,2023-10-17T19:15:16.000Z,None,None,False,,public,en,https://hachyderm.io/users/Brahn/statuses/1112...,https://hachyderm.io/@Brahn/111251955150490629,...,"<p><a href=""https://flotes.app/"" rel=""nofollow...",None,"{'id': '109407218535499997', 'username': 'Brah...",[],[],"[{'name': 'notes', 'url': 'https://hci.social/...",[],"{'url': 'https://flotes.app/', 'title': 'Flote...",None,NaN
4,111251955053018059,2023-10-17T19:15:12.000Z,111251953522801138,109327309439812369,False,,public,en,https://androiddev.social/users/MishaalRahman/...,https://androiddev.social/@MishaalRahman/11125...,...,<p>Two things that are missing from AOSP curre...,None,"{'id': '109327309439812369', 'username': 'Mish...",[],[],[],[],{'url': 'https://www.androidauthority.com/pixe...,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,111248236137466141,2023-10-17T03:29:23.760Z,None,None,False,,public,None,https://stop.voring.me/notes/9kxbxvlci7wim97b,https://stop.voring.me/notes/9kxbxvlci7wim97b,...,<p><span>This bunny has been seen 83 times.<br...,None,"{'id': '109287795052914698', 'username': 'bunn...","[{'id': '111248235995612185', 'type': 'gifv', ...",[],"[{'name': 'Bot', 'url': 'https://hci.social/ta...",[],"{'url': 'https://www.bunnies.io/#150', 'title'...",None,NaN
36,111248235643330355,2023-10-17T03:29:21.000Z,111248233104718931,109502881445583957,False,,public,en,https://infosec.exchange/users/cannibal/status...,https://infosec.exchange/@cannibal/11124823568...,...,"<p>I started with the finest grade copper 3/4""...",None,"{'id': '109502881445583957', 'username': 'cann...","[{'id': '111248379224984643', 'type': 'image',...",[],[],[],None,None,NaN
37,111248235402508488,2023-10-17T03:29:17.000Z,None,None,False,,public,en,https://laserdisc.party/users/checkervest/stat...,https://laserdisc.party/@checkervest/111248235...,...,<p>If you watched Fall of the House of Usher o...,None,"{'id': '108402464028726003', 'username': 'chec...",[],[],[],[],None,None,NaN
38,111248235217293137,2023-10-17T03:29:13.000Z,111248116859074804,109357144467018121,False,,public,en,https://mastodon.social/users/dungeons/statuse...,https://mastodon.social/@dungeons/111248235126...,...,<p>You were hit by the Green Dragon Wyrmling's...,None,"{'id': '109357144467018121', 'username': 'dung...",[],[],[],[],None,"{'id': '76160', 'expires_at': '2023-10-17T03:5...",NaN


hmm, I think I might have just gotten booted off? and how do we prevent this from breaking when the server burps?